In [1]:
import pandas as pd
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi, HfFolder
import os

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

# Set the token for use in the huggingface_hub library
HfFolder.save_token(hf_token)

import wandb

# Replace 'your-api-token' with your actual API token from wandb.ai
wandb.login(key='88bc0f92c0138587605140040c6d0ef652bde63d')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
df = pd.read_csv("/kaggle/input/mental-health-corpus/mental_health.csv")

In [3]:
df

,text,label
0,dear american teens question dutch person hear...,0
1,nothing look forward lifei dont many reasons k...,1
2,music recommendations im looking expand playli...,0
3,im done trying feel betterthe reason im still ...,1
4,worried year old girl subject domestic physic...,1
...,...,...
27972,posting everyday people stop caring religion ...,0
27973,okay definetly need hear guys opinion ive pret...,0
27974,cant get dog think ill kill myselfthe last thi...,1
27975,whats point princess bridei really think like ...,1


In [4]:
df2 = pd.read_csv("/kaggle/input/mental-health-social-media/Mental-Health-Twitter.csv")

In [5]:
df2 = df2[["post_text", "label"]]

In [6]:
df2.rename(columns={'post_text': 'text'}, inplace=True)

In [7]:
df = pd.concat([df, df2], ignore_index=True)

In [8]:
df

,text,label
0,dear american teens question dutch person hear...,0
1,nothing look forward lifei dont many reasons k...,1
2,music recommendations im looking expand playli...,0
3,im done trying feel betterthe reason im still ...,1
4,worried year old girl subject domestic physic...,1
...,...,...
47972,A day without sunshine is like night.,0
47973,"Boren's Laws: (1) When in charge, ponder. (2) ...",0
47974,The flow chart is a most thoroughly oversold p...,0
47975,"Ships are safe in harbor, but they were never ...",0


In [9]:
df = pd.get_dummies(df, columns=["label"])
df["label_0"] = df["label_0"].astype(int)
df["label_1"] = df["label_1"].astype(int)

In [10]:
X_data = df["text"]
y_data = df[["label_0", "label_1"]]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.3, stratify=y_data, random_state=42
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (33583,)
y_train shape: (33583, 2)
X_test shape: (14394,)
y_test shape: (14394, 2)


In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
def preprocess_function(examples):
    #print(examples)
    return tokenizer(str(examples), truncation=True, padding="max_length", max_length=512)

In [14]:
X_train = X_train.map(preprocess_function)
X_test = X_test.map(preprocess_function)

In [15]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

2024-03-30 22:01:20.204509: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-30 22:01:20.204646: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-30 22:01:20.435147: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [16]:
%pip install evaluate
import evaluate

accuracy = evaluate.load("accuracy")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 752.4 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [18]:
id2label = {0: "POSITIVE", 1: "NEGATIVE"}
label2id = {"POSITIVE": 0, "NEGATIVE": 1}
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from datasets import Dataset
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)
X_train = pd.DataFrame(X_train.tolist(), columns=['input_ids', 'attention_mask'], index=X_train.index)
X_test = pd.DataFrame(X_test.tolist(), columns=['input_ids', 'attention_mask'], index=X_test.index)


In [20]:
X_train

,input_ids,attention_mask
36808,"[101, 1030, 3335, 7393, 25688, 1045, 2123, 100...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
38449,"[101, 1030, 7232, 9021, 4939, 2074, 2994, 2039...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."
40051,"[101, 19387, 1030, 8869, 20464, 12069, 1024, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
37851,"[101, 2123, 1005, 1056, 2104, 4355, 21499, 100...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
28944,"[101, 1030, 9318, 1035, 20996, 2243, 8713, 223...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
...,...,...
17269,"[101, 2215, 3280, 5714, 4013, 9215, 6069, 3102...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
29430,"[101, 1030, 11688, 2080, 16932, 2581, 2052, 20...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
16002,"[101, 8239, 3674, 8040, 3917, 12650, 21572, 36...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
636,"[101, 2253, 5010, 7910, 2209, 3067, 10419, 102...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [21]:
train_dataset = Dataset.from_dict({
    "input_ids": X_train['input_ids'],
    "attention_mask": X_train['attention_mask'],
    "labels": y_train
})

test_dataset = Dataset.from_dict({
    "input_ids": X_test['input_ids'],
    "attention_mask": X_test['attention_mask'],
    "labels": y_test
})

In [22]:
#from transformers import DataCollatorWithPadding
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

training_args = TrainingArguments(
    output_dir="Mental_health_identification",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Currently logged in as: julien-serbanescu (juteam). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.211300,0.194451,0.920592
2,0.129800,0.183144,0.932124


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=2100, training_loss=0.20248587926228842, metrics={'train_runtime': 2026.3228, 'train_samples_per_second': 33.147, 'train_steps_per_second': 1.036, 'total_flos': 8897305298128896.0, 'train_loss': 0.20248587926228842, 'epoch': 2.0})

In [23]:
trainer.push_to_hub()

events.out.tfevents.1711836127.41ee23786abc.34.0:   0%|          | 0.00/6.47k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DaJulster/Mental_health_identification/commit/0bf565ff1f5951ca7d0a22795f6e2eaf754bb9cb', commit_message='End of training', commit_description='', oid='0bf565ff1f5951ca7d0a22795f6e2eaf754bb9cb', pr_url=None, pr_revision=None, pr_num=None)